<a href="https://colab.research.google.com/github/kingajutrzenka/ML-Bootcamp-I/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Przygotowanie danych do modelu**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [3]:
data = {
    'size': ['XL', 'L', 'M', 'L', 'M'],
    'color': ['red', 'green', 'blue', 'green', 'red'],
    'gender': ['female', 'male', 'male', 'female', 'female'],
    'price': [199.0, 89.0, 99.0, 129.0, 79.0],
    'weight': [500, 450, 300, 380, 410],
    'bought': ['yes', 'no', 'yes', 'no', 'yes']
}

df_raw = pd.DataFrame(data=data)
df_raw


,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


In [4]:
df = df_raw.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   size    5 non-null      object 
 1   color   5 non-null      object 
 2   gender  5 non-null      object 
 3   price   5 non-null      float64
 4   weight  5 non-null      int64  
 5   bought  5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 372.0+ bytes


In [5]:
# zmiana typu danych
for col in ['size', 'color', 'gender', 'bought']:
    df[col] = df[col].astype('category') #nowy typ category

df['weight'] = df['weight'].astype('float') # zmiana typu int -> float

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   size    5 non-null      category
 1   color   5 non-null      category
 2   gender  5 non-null      category
 3   price   5 non-null      float64 
 4   weight  5 non-null      float64 
 5   bought  5 non-null      category
dtypes: category(4), float64(2)
memory usage: 744.0 bytes


In [6]:
df.describe()

,price,weight
count,5.000000,5.000000
mean,119.000000,408.000000
std,48.476799,75.299402
min,79.000000,300.000000
25%,89.000000,380.000000
50%,99.000000,410.000000
75%,129.000000,450.000000
max,199.000000,500.000000


In [7]:
# Pokazuje statystyki dla kolumn typu category
df.describe(include=['category']).T

,count,unique,top,freq
size,5,3,L,2
color,5,3,green,2
gender,5,2,female,3
bought,5,2,yes,3


In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['bought'])
le.transform(df['bought'])

# zmiana na 0-1
# yes - 1
# no - 0
# krótszy zapis : le.fit_transform(df['bought'])

array([1, 0, 1, 0, 1])

In [11]:
le.classes_ #pokazuje jak jest mapowane

array(['no', 'yes'], dtype=object)

In [12]:
df['bought'] = le.fit_transform(df['bought'])
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,1
1,L,green,male,89.0,450.0,0
2,M,blue,male,99.0,300.0,1
3,L,green,female,129.0,380.0,0
4,M,red,female,79.0,410.0,1


In [13]:
le.inverse_transform(df['bought']) #wracamy do pierwotnej postaci

array(['yes', 'no', 'yes', 'no', 'yes'], dtype=object)

metoda wyżej raczej do zmiennej docelowej niż do innych

In [19]:
# Apply OneHotEncoder to the 'size' column
size_encoded = encoder.transform(df[['size']])
display(size_encoded)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5 stored elements and shape (5, 3)>

In [20]:
# Apply OneHotEncoder to 'color' and 'gender' columns
# We need a new encoder for these columns as the categories are different
encoder_color_gender = OneHotEncoder()
color_gender_encoded = encoder_color_gender.fit_transform(df[['color', 'gender']])
display(color_gender_encoded)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10 stored elements and shape (5, 5)>

In [21]:
encoder.categories_

[array(['L', 'M', 'XL'], dtype=object)]

better:

# Co robić

In [25]:
pd.get_dummies(data=df, drop_first=True, dtype = 'int',
               prefix_sep='-', prefix = 'new')

,price,weight,bought,new-M,new-XL,new-green,new-red,new-male
0,199.0,500.0,1,0,1,0,1,0
1,89.0,450.0,0,0,0,1,0,1
2,99.0,300.0,1,1,0,0,0,1
3,129.0,380.0,0,0,0,1,0,0
4,79.0,410.0,1,1,0,0,1,0


In [26]:
pd.get_dummies(data=df, drop_first=True, dtype = 'int',
               prefix_sep='-', prefix = 'new',
               columns=['size'])
# zmienia tylko  wybraną kolumnę

,color,gender,price,weight,bought,new-M,new-XL
0,red,female,199.0,500.0,1,0,1
1,green,male,89.0,450.0,0,0,0
2,blue,male,99.0,300.0,1,1,0
3,green,female,129.0,380.0,0,0,0
4,red,female,79.0,410.0,1,1,0


# Zmienne numeryczne (standaryzacja)

price i weight

to to samo:

In [27]:
def standarize(x):
  return (x - x.mean()) / x.std()

standarize(df['price'])

,price
0,1.650274
1,-0.618853
2,-0.412568
3,0.206284
4,-0.825137


In [28]:
from sklearn.preprocessing import scale

scale(df['price'])

array([ 1.84506242, -0.69189841, -0.4612656 ,  0.2306328 , -0.92253121])

In [31]:
# ta zachowuje:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['price', 'weight']] = scaler.fit_transform(df[['price', 'weight']])

In [32]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,1.845062,1.366002,1
1,L,green,male,-0.691898,0.623610,0
2,M,blue,male,-0.461266,-1.603567,1
3,L,green,female,0.230633,-0.415740,0
4,M,red,female,-0.922531,0.029696,1


# Cały mechanizm po kolei

In [33]:
df = df_raw.copy()
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


In [34]:
le = LabelEncoder()
df['bought'] = le.fit_transform(df['bought'])

scaler = StandardScaler()
df[['price', 'weight']] = scaler.fit_transform(df[['price', 'weight']])

df = pd.get_dummies(data=df, drop_first=True, dtype = 'int')
df

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_male
0,1.845062,1.366002,1,0,1,0,1,0
1,-0.691898,0.623610,0,0,0,1,0,1
2,-0.461266,-1.603567,1,1,0,0,0,1
3,0.230633,-0.415740,0,0,0,1,0,0
4,-0.922531,0.029696,1,1,0,0,1,0
